In [1]:
#pip install pymongo

In [2]:
import numpy as np
import pymongo
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split




In [3]:
client = pymongo.MongoClient('localhost:27017')

db = client['movie']

movies = db['movie_json']
users = db['user_json']

In [4]:
distinct_genres = movies.aggregate([
    {
        "$project": {
            "genres": {"$split": ["$genres", "|"]}
        }
    },{
        "$unwind": "$genres"
    },
    {
        "$group": {
            "_id": "$genres",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
])

[genre for genre in distinct_genres]

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65cb48c8d80eb9a5b310e04f, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [5]:
desired_genres = ['Comedy', 'Horror']
filtered_movies = movies.find({"genres": {"$regex": f".*{'|'.join(desired_genres)}.*"}})

In [ ]:
list(filtered_movies)

In [ ]:
movies_ids = movies.find({'genres': {'$regex': 'Comedy|Horror'}}).distinct('_id')
users_ids = users.distinct('_id')

In [ ]:
df = pd.DataFrame(0, index=users_ids, columns=movies_ids)
df
# a l'aide d'un produit scalaire 

In [ ]:
films = ['film1', 'film2', 'film3']

In [ ]:
idx = 0
for f in films:
    print(idx, f)
    idx += 1

In [ ]:
for idx, f in enumerate(films):
    print(idx, f)

In [ ]:
for u_id in users_ids:
    user = users.find_one({'_id': u_id})

    for m in user['movies']:
        
        m_id = m['movieid']
        rating = m['rating']

        if m_id in movies_ids:
            df.loc[u_id, m_id] = rating

In [ ]:
df

In [ ]:
df = df[df.sum(axis=1) > 0]
df

In [ ]:
df_sparse = df.astype(pd.SparseDtype("float", 0))

In [ ]:
nmf = NMF(n_components=2)

# Fit the model to the user-item matrix
nmf.fit(df_sparse)

U = nmf.transform(df_sparse)  # User matrix
M = nmf.components_  # Item matrix

pred_matrix = np.dot(U, M)

print(mean_squared_error(df_sparse.values, pred_matrix))

In [ ]:
movies_genres = {}

for m_id in movies_ids:
    cur_movie = movies.find_one({'_id': m_id}, {'genres': 1})
    cur_genres = cur_movie['genres']  # Utilisation de crochets [] pour accéder aux valeurs de la clé
    if 'Comedy' in cur_genres and 'Horror' in cur_genres:
        movies_genres[m_id] = 'red'
    elif 'Comedy' in cur_genres:
        movies_genres[m_id] = 'lightgreen'
    else:
        movies_genres[m_id] = 'black'

movies_genres


In [ ]:
# plt.scatter(M[0, :], M[1,:], c=movies_genres.values())
import matplotlib.pyplot as plt
import mplcursors

# Tracer le nuage de points avec les couleurs correspondant aux genres
scatter = plt.scatter(M[0, :], M[1, :], c=list(movies_genres.values()))

# Définir les informations à afficher lors du survol
@scatter.tooltip
def on_hover(sel):
    index = sel.target.index
    movie_id = movies_ids[index]
    movie_title = movies.find_one({'_id': movie_id}, {'title': 1})['title']
    movie_genre = list(movies_genres.values())[index]
    return f"Film : {movie_title}\nGenre : {movie_genre}"

# Afficher le graphique
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Scatter plot avec infos sur le hover')
mplcursors.cursor(hover=True)
plt.show()

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Supposons que vous avez déjà chargé vos données dans des variables comme suit :
# X_train, X_test, y_train, y_test = ...

# Commencez un nouveau run MLflow
with mlflow.start_run():

    # Définir les paramètres de la run
    mlflow.log_param("model", "RandomForestRegressor")

    # Créer et entraîner le modèle
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Faire des prédictions sur l'ensemble de test
    predictions = model.predict(X_test)

    # Calculer et enregistrer la métrique MSE
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "random_forest_model")

# Fin de la run MLflow



In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Supposons que vous avez déjà chargé vos données dans des variables comme suit :
# X_train, X_test, y_train, y_test = ...

# Commencez un nouveau run MLflow
with mlflow.start_run():

    # Définir les paramètres de la run
    mlflow.log_param("model", "RandomForestRegressor")

    # Créer et entraîner le modèle
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Faire des prédictions sur l'ensemble de test
    predictions = model.predict(X_test)

    # Calculer et enregistrer la métrique MSE
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)

    # Enregistrer le modèle
    mlflow.sklearn.log_model(model, "random_forest_model")

# Fin de la run MLflow
